In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from datetime import datetime

In [23]:
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.ensemble import VotingClassifier
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split
import pickle
import random
from sklearn.metrics import accuracy_score,f1_score,classification_report
from sklearn.neighbors import KNeighborsClassifier

from tqdm import tqdm
import sys
import os
import warnings
if not sys.warnoptions:
	warnings.simplefilter("ignore")							#1
	os.environ["PYTHONWARNINGS"] = "ignore"					#2

	warnings.filterwarnings('ignore', 'Solver terminated early.*')		#3
	warnings.filterwarnings('ignore', category=ConvergenceWarning)	#4

In [24]:
# 데이터 읽기
train = pd.read_csv('./2023_ML_프로젝트_데이터/Train.csv')
test = pd.read_csv('./2023_ML_프로젝트_데이터/Test.csv')

In [25]:
# index drop
train = train.drop('Index',axis=1)
test = test.drop('Index',axis=1)

In [26]:
# train용 데이터의 독립변수와 종속변수의 분리
X = train.loc[:,:'X999']
y = train.iloc[:,-1]

In [27]:
# score 를 보기 위해 split, 제출용은 Full Train으로 제작할 것
def test_smote_model(i):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=22)
    smote = SMOTE(random_state=i)
    X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)

    # 앙상블 방식
    classifiers = {}
    classifiers1 = RandomForestClassifier(n_estimators=30)
    classifiers2 = LogisticRegression(max_iter=20)
    # classifiers2 = GradientBoostingClassifier(n_estimators=30)
    classifiers3 = KNeighborsClassifier()
    # classifiers3 = AdaBoostClassifier(n_estimators = 30)

    model = VotingClassifier(estimators=[('lr',classifiers1),('rf',classifiers2),('gnb',classifiers3)],voting = 'soft',weights=[5,1,1])
    ensemble_train = model.fit(X_train_resampled,y_train_resampled)

    ensemble_pred = model.predict(X_test)
    score = f1_score(y_test,ensemble_pred,pos_label='positive')
    if round(score,2) >= 0.5:
        with open('model/smote_test_{}.pkl'.format(round(score,4)),'wb') as f:
            pickle.dump(model,f)
    return round(score,4)
    # print("Classificcation Report: \n{}".format(classification_report(y_test,ensemble_pred)))


In [28]:
for i in tqdm(range(20)):
    a = test_smote_model(i)
    print(a)


  5%|▌         | 1/20 [00:06<02:12,  6.97s/it]

0.4384


 10%|█         | 2/20 [00:14<02:10,  7.23s/it]

0.4615


 15%|█▌        | 3/20 [00:21<02:02,  7.23s/it]

...

0.4667


 95%|█████████▌| 19/20 [02:18<00:07,  7.32s/it]

0.4552


100%|██████████| 20/20 [02:25<00:00,  7.27s/it]

0.469


In [29]:
with open('model/smote_test_0.5.pkl','rb') as f:
    model = pickle.load(f)

In [30]:
# model test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=22)
smote = SMOTE(random_state=i)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)

# 앙상블 방식
classifiers = {}
classifiers1 = RandomForestClassifier(n_estimators=30)
classifiers2 = LogisticRegression(max_iter=20)
# classifiers2 = GradientBoostingClassifier(n_estimators=30)
classifiers3 = KNeighborsClassifier()
# classifiers3 = AdaBoostClassifier(n_estimators = 30)
# ensemble_train = model.fit(X_train_resampled,y_train_resampled)

ensemble_pred = model.predict(X_test)
score = f1_score(y_test,ensemble_pred,pos_label='positive')

In [35]:
ensemble_pred_for_csv = model.predict(test.loc[:,:'X999'])

print(f'Type : {type(ensemble_pred_for_csv)} | Length : {len(ensemble_pred_for_csv)}')

import numpy as np

np.savetxt('./버킷햇_0606.csv', ensemble_pred_for_csv, delimiter=',',fmt='%s')

Type : <class 'numpy.ndarray'> | Length : 3500


In [13]:
from sklearn.metrics import f1_score
# print(classification_report(y_test,ensemble_pred))
# print(accuracy_score(y_test,ensemble_pred))
print(f1_score(y_test,ensemble_pred,pos_label='negative',average=None))

[0.96137339 0.47058824]


c:\Users\alseh\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1396: UserWarning: Note that pos_label (set to 'negative') is ignored when average != 'binary' (got None). You may use labels=[pos_label] to specify a single positive class.
  warnings.warn(


TypeError: 'NoneType' object is not subscriptable

In [ ]:
# 제출에 사용할 결과
ensemble_pred_for_csv = ensemble_train.predict(test.loc[:,:'X999'])

In [31]:
#logistice 버전
# Assuming you have your feature data in X and target data in y
# X is a 2D array with shape (5000, num_features)
# y is a 1D array with shape (5000,)

# Split the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=22)

# Apply SMOTE to oversample the minority class
smote = SMOTE(random_state=42)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)

# Train a classification model (e.g., logistic regression)
model = LogisticRegression()
model.fit(X_train_resampled, y_train_resampled)

# Predict on the test set
y_pred = model.predict(X_test)
# Calculate the F1-score
# f1score = classification_report(y_test, y_pred)
print("Classificcation Report: \n{}".format(classification_report(y_test, y_pred)))
# print(f1score)


Classificcation Report: 
              precision    recall  f1-score   support

    negative       0.97      0.89      0.93       925
    positive       0.33      0.67      0.44        75

    accuracy                           0.88      1000
   macro avg       0.65      0.78      0.69      1000
weighted avg       0.92      0.88      0.89      1000



c:\Users\alseh\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [8]:
type(y_pred)

numpy.ndarray

In [11]:
test_ = test.loc[:,'X0':'X999']

In [12]:
type(y_pred)

numpy.ndarray

In [13]:
y_pred_ = model.predict(test_)
print(f'Type : {type(y_pred_)} | Length : {len(y_pred_)}')

Type : <class 'numpy.ndarray'> | Length : 3500


In [14]:
import numpy as np

np.savetxt('./버킷햇_0606.csv', y_pred_, delimiter=',',fmt='%s')

In [74]:
import numpy as np
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import classification_report
# svc 버전
# Assuming you have your feature data in X and target data in y
# X is a 2D array with shape (5000, num_features)
# y is a 1D array with shape (5000,)

# Split the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Apply SMOTE to oversample the minority class
smote = SMOTE(random_state=42)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)

# Train an SVM model
model = SVC()
model.fit(X_train_resampled, y_train_resampled)

# Predict on the test set
y_pred = model.predict(X_test)

# Calculate the F1-score
# f1score = classification_report(y_test, y_pred)
print("Classificcation Report: \n{}".format(classification_report(y_test, y_pred)))
# print(f1score)


Classificcation Report: 
              precision    recall  f1-score   support

    negative       0.95      0.86      0.90       902
    positive       0.30      0.56      0.39        98

    accuracy                           0.83      1000
   macro avg       0.62      0.71      0.64      1000
weighted avg       0.88      0.83      0.85      1000



In [33]:
len(y_pred)

3500

In [9]:
train.target.value_counts()

negative    4564
positive     436
Name: target, dtype: int64